In [ ]:
############ IMPORTS ####################

# import sys
# sys.path.insert(0,'/home/users/jgecvision/.conda/envs/ankur01/lib/python3.8/site-packages')
import os
import numpy as np
import torch.utils.data as dataf
from scipy import io
import sys
import spectral
import tensorflow as tf
from sklearn.decomposition import PCA
sys.path.append("./../")
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import torch
from operator import truediv
import record
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.engine.topology import Layer
from tensorflow.keras import initializers, constraints
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from keras.utils import conv_utils
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import InputSpec, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Reshape, BatchNormalization, Layer, Dropout
from tensorflow.keras.layers import Dropout, Input, LeakyReLU, Multiply
from tensorflow.keras.layers import GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.layers import Add, Activation, GlobalMaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical as keras_to_categorical


############# CONFIGS ##########################

# tf.config.experimental_run_functions_eagerly(True)
# os.environ["CUDA_VISIBLE_DEVICES"]="3"

datasetNames = ["Trento"]
testSizeNumber = 5000
patchsize1 = 11
patchsize2 = 11
batchsize = 64
EPOCH = 200
LR = 0.001

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100




for datasetName in datasetNames:
       
        print("----------------------------------Training for ",datasetName,"---------------------------------------------")

        try:
            os.makedirs(datasetName)
        except FileExistsError:
            pass
        # Train data
        HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
        TrainPatch = HSI['Data']
        TrainPatch = TrainPatch.astype(np.float32)
        NC = TrainPatch.shape[3] # NC is number of bands

        label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
        TrLabel = label['Data']

        # Test data
        HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
        TestPatch = HSI['Data']
        TestPatch = TestPatch.astype(np.float32)

        label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
        TsLabel = label['Data']


        TrainPatch1 = torch.from_numpy(TrainPatch)
    #         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
        TrainLabel1 = torch.from_numpy(TrLabel)-1
        TrainLabel1 = TrainLabel1.long()


        TestPatch1 = torch.from_numpy(TestPatch)
    #         TestPatch1 = TestPatch1.permute(0,3,1,2)
        TestLabel1 = torch.from_numpy(TsLabel)-1
        TestLabel1 = TestLabel1.long()

        Classes = len(np.unique(TrainLabel1))
        TrainPatch1 = TrainPatch1.reshape(TrainPatch1.shape[0],TrainPatch1.shape[1]*TrainPatch1.shape[2],TrainPatch1.shape[3])
        TestPatch1 = TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1]*TestPatch1.shape[2],TestPatch1.shape[3])
        print("Train data shape = ", TrainPatch1.shape)
        print("Train label shape = ", TrainLabel1.shape)
        print("Test data shape = ", TestPatch1.shape)
        print("Test label shape = ", TestLabel1.shape)
        
        
        
        KAPPA = []
        OA = []
        AA = []
        ELEMENT_ACC = np.zeros((3, Classes))
        tf.compat.v1.keras.backend.clear_session()
        config = tf.compat.v1.ConfigProto( device_count = {'GPU': 0} ) 
        config.gpu_options.allow_growth = True
        sess = tf.compat.v1.Session(config=config) 
        tf.compat.v1.keras.backend.set_session(sess)
        g = tf.Graph()
        with g.as_default():
            filters = 64
            class Erosion2D(Layer):
                '''
                Erosion 2D Layer
                for now assuming channel last
                '''

                def __init__(self, num_filters, kernel_size, strides=(1, 1),
                             padding='same', kernel_initializer='glorot_uniform',
                             kernel_constraint=None,
                             **kwargs):
                    super(Erosion2D, self).__init__(**kwargs)
                    self.num_filters = num_filters
                    self.kernel_size = kernel_size
                    self.strides = strides
                    self.padding = padding

                    self.kernel_initializer = initializers.get(kernel_initializer)
                    self.kernel_constraint = constraints.get(kernel_constraint)
                    # for we are assuming channel last
                    self.channel_axis = -1

                    # self.output_dim = output_dim

                def build(self, input_shape):
                    if input_shape[self.channel_axis] is None:
                        raise ValueError('The channel dimension of the inputs '
                                         'should be defined. Found `None`.')

                    input_dim = input_shape[self.channel_axis]
                    kernel_shape = self.kernel_size + (input_dim, self.num_filters)

                    self.kernel = self.add_weight(shape=kernel_shape,
                                                  initializer=self.kernel_initializer,
                                                  name='kernel',
                                                  constraint=self.kernel_constraint)

                    # Be sure to call this at the end
                    super(Erosion2D, self).build(input_shape)
                    self.input_spec = InputSpec(ndim=4,
                                                axes={3: input_shape[-1]})
                    self.built = True

                def call(self, x):
                    outputs = self.erosion2d(x, self.kernel[...,1],
                                             self.strides, self.padding)

                    return outputs

                def erosion2d(self, x, st_element, strides, padding,
                                rates=(1, 1, 1, 1)):
                    # tf.nn.erosion2d(input, filter, strides, rates, padding, name=None)
                    x = tf.compat.v1.nn.erosion2d(x, st_element, (1, ) + strides + (1, ),
                                        rates, padding.upper())
                    return x

            class Dilation2D(Layer):
                '''
                Dilation 2D Layer
                for now assuming channel last
                '''

                def __init__(self, num_filters, kernel_size, strides=(1, 1),
                             padding='same', kernel_initializer='glorot_uniform',
                             kernel_constraint=None,
                             **kwargs):
                    super(Dilation2D, self).__init__(**kwargs)
                    self.num_filters = num_filters
                    self.kernel_size = kernel_size
                    self.strides = strides
                    self.padding = padding

                    self.kernel_initializer = initializers.get(kernel_initializer)
                    self.kernel_constraint = constraints.get(kernel_constraint)
                    # for we are assuming channel last
                    self.channel_axis = -1

                    # self.output_dim = output_dim

                def build(self, input_shape):
                    if input_shape[self.channel_axis] is None:
                        raise ValueError('The channel dimension of the inputs '
                                         'should be defined. Found `None`.')

                    input_dim = input_shape[self.channel_axis]
                    kernel_shape = self.kernel_size + (input_dim, self.num_filters)

                    self.kernel = self.add_weight(shape=kernel_shape,
                                                  initializer=self.kernel_initializer,
                                                  name='kernel',
                                                  constraint=self.kernel_constraint)

                    # Be sure to call this at the end
                    super(Dilation2D, self).build(input_shape)
                    self.input_spec = InputSpec(ndim=4,
                                                axes={3: input_shape[-1]})
                    self.built = True

                def call(self, x):
                    outputs = self.dilation2d(x, self.kernel[...,1],
                                             self.strides, self.padding)

                    return outputs

                def dilation2d(self, x, st_element, strides, padding,
                                rates=(1, 1, 1, 1)):
                    # tf.nn.erosion2d(input, filter, strides, rates, padding, name=None)
                    x = tf.compat.v1.nn.dilation2d(x, st_element, (1, ) + strides + (1, ),
                                        rates, padding.upper())
                    return x


            class SpectralMorph(Layer):
                def __init__(self):

                    super(SpectralMorph, self).__init__()
                    self.filters = filters
                    num_filters = filters
                    #self.init = RandomNormal()
                def call(self, x):
                    z1 = Erosion2D(num_filters = self.filters, kernel_size = (3,3),padding="same", strides=(1,1))(x)
                    z1 = Conv2D(filters = self.filters, kernel_size = (1,1), padding='same')(z1)
                    z2 = Dilation2D(num_filters = self.filters, kernel_size = (3,3), padding="same", strides=(1,1))(x)
                    z2 = Conv2D(filters = self.filters, kernel_size = (1,1), padding='same')(z2)
                    x = Add()([z1, z2])
                    return x


            class SpatialMorph(Layer):
                def __init__(self):

                    super(SpatialMorph, self).__init__()
                    self.filters = filters
                    num_filters = filters
                    #self.init = RandomNormal()
                def call(self, x):
                    z1 = Erosion2D(num_filters = self.filters, kernel_size = (3,3),padding="same", strides=(1,1))(x)
                    z1 = Conv2D(filters = self.filters, kernel_size = (3,3), padding='same')(z1)
                    z2 = Dilation2D(num_filters = self.filters, kernel_size = (3,3), padding="same", strides=(1,1))(x)
                    z2 = Conv2D(filters = self.filters, kernel_size = (3,3), padding='same')(z2)
                    x = Add()([z1, z2])
                    return x




            for iter in range(3):
        #             tf.compat.v1.set_random_seed(43)
        #             np.random.seed(43)

                input_layer = Input((11, 11,NC))

                z = Conv2D(filters, use_bias = True, kernel_size = 3, padding = 'same')(input_layer)
                z = BatchNormalization()(z)
                z = Activation('relu')(z)

                z = SpectralMorph()(z)
                z = BatchNormalization()(z)
                z = Activation('relu')(z)

                z = Conv2D(64, use_bias = True, kernel_size = 3, padding = 'same')(z)
                z = BatchNormalization()(z)
                z = Activation('relu')(z)


                z = SpatialMorph()(z)
                z = BatchNormalization()(z)
                z = Activation('relu')(z)

    #             z = MaxPooling2D(pool_size=(2,2), strides =1, padding = 'same')(z)

                z = Conv2D(128, use_bias = True, kernel_size = 3, padding = 'same')(z)
                z = BatchNormalization()(z)
                z = Activation('relu')(z)
    #             z = keras.layers.Concatenate(axis=3)([zspec,zspat])

                z = MaxPooling2D(pool_size=(2,2), strides =2, padding = 'same')(z)

                z = Conv2D(512, use_bias = True, kernel_size = 3, padding = 'same')(z)
                z = BatchNormalization()(z)
                z = Activation('relu')(z)
                #z = Dropout(0.5)(z)

                #z = Flatten()(z)
                z = GlobalAveragePooling2D()(z)
                #z = Dropout(0.3)(z)
                z = Dense(128, activation="relu")(z)
                z = BatchNormalization()(z)
                z = Dropout(0.5)(z)
                z = Dense(64, activation="relu")(z)
                z = BatchNormalization()(z)
                z = Dropout(0.5)(z)
                output_layer = Dense(Classes, activation="softmax")(z)




                clf = Model(inputs=input_layer, outputs=output_layer)
                clf.summary()
                clf.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])


                valdata = (TestPatch1.cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
                h = clf.fit(TrainPatch1.cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                                    batch_size=batchsize,
                                    epochs=EPOCH,
                                    verbose=True,
                                    validation_data=valdata,
                                    workers=4,
                                    callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly.h5", monitor='val_accuracy', verbose=0, save_best_only=True,save_weights_only=True)])



                clf.load_weights(datasetName+"/best_model_HSIOnly.h5")
                pred_y = np.argmax(clf.predict(TestPatch1.cpu().detach().numpy()), axis=1)

                y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
                oa = accuracy_score(y_test, pred_y)*100
                confusion = confusion_matrix(y_test, pred_y)
                each_acc, aa = AA_andEachClassAccuracy(confusion)
                kappa = cohen_kappa_score(y_test, pred_y)*100
                KAPPA.append(kappa)
                OA.append(oa)
                AA.append(aa)
                ELEMENT_ACC[iter, :] = each_acc

        print("--------" + datasetName + " Training Finished-----------")
        record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/MorphCNN_Report_' + datasetName +'.txt')


2021-11-11 03:40:36.060458: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.2


Trento
True
(819, 11, 11, 63)
Test patch 1 shape  torch.Size([29395, 11, 11, 63])
6

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2021-11-11 03:40:39.359299: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3783000000 Hz
2021-11-11 03:40:39.366322: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x148881220 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-11 03:40:39.366350: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-11 03:40:39.370349: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-11 03:40:39.399778: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1105] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-11-11 03:40:39.399798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1111]      
2021-11-11 03:40:39.466478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1564] Found device 0 with properties: 
pciBusID: 0035:04:00.0 name: Tesla V100-SXM2-32GB computeCapabilit

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 11, 11, 63)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 11, 11, 64)        36352     
_________________________________________________________________
batch_normalization (BatchNo (None, 11, 11, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 11, 11, 64)        0         
_________________________________________________________________
spectral_morph (SpectralMorp (None, 11, 11, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 64)        0     

2021-11-11 03:40:42.212697: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14a497b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-11 03:40:42.212828: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2021-11-11 03:40:42.215144: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1564] Found device 0 with properties: 
pciBusID: 0035:04:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.50GiB deviceMemoryBandwidth: 836.37GiB/s
2021-11-11 03:40:42.215177: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.2
2021-11-11 03:40:42.215193: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-11 03:40:42.215206: I tensorflow/stream_executor/platform/default/dso_load

Epoch 1/200
819/819 [==============================] - 11s 13ms/sample - loss: 1.0501 - accuracy: 0.6654 - val_loss: 1.6717 - val_accuracy: 0.4547
Epoch 2/200
819/819 [==============================] - 9s 11ms/sample - loss: 0.3356 - accuracy: 0.9170 - val_loss: 1.5281 - val_accuracy: 0.5744
Epoch 3/200
819/819 [==============================] - 9s 11ms/sample - loss: 0.2160 - accuracy: 0.9475 - val_loss: 1.2191 - val_accuracy: 0.4414
Epoch 4/200
819/819 [==============================] - 9s 11ms/sample - loss: 0.1403 - accuracy: 0.9634 - val_loss: 1.2573 - val_accuracy: 0.5101
Epoch 5/200
819/819 [==============================] - 9s 11ms/sample - loss: 0.0801 - accuracy: 0.9878 - val_loss: 1.7978 - val_accuracy: 0.2250
Epoch 6/200
819/819 [==============================] - 12s 14ms/sample - loss: 0.0538 - accuracy: 0.9939 - val_loss: 2.0616 - val_accuracy: 0.2258
Epoch 7/200
819/819 [==============================] - ETA: 0s - loss: 0.0506 - accuracy: 0.9927

In [2]:
 clf.load_weights(datasetName+"/best_model_HSIOnly_Deviation.h5")

2021-11-11 06:11:58.642248: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1564] Found device 0 with properties: 
pciBusID: 0035:04:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.50GiB deviceMemoryBandwidth: 836.37GiB/s
2021-11-11 06:11:58.661769: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.2
2021-11-11 06:11:58.667346: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-11 06:11:58.667395: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-11-11 06:11:58.667419: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2021-11-11 06:11:58.758397: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolv

TypeError: An op outside of the function building code is being passed
a "Graph" tensor. It is possible to have Graph tensors
leak out of the function building context by including a
tf.init_scope in your function building code.
For example, the following function will fail:
  @tf.function
  def has_init_scope():
    my_constant = tf.constant(1.)
    with tf.init_scope():
      added = my_constant * 2
The graph tensor has name: conv2d_16/kernel:0

In [10]:
with g.as_default():
    clf.save_weights(datasetName+"/best_model_HSIOnly_Deviation.h5")

In [8]:
with g.as_default():
    clf.save(datasetName+"/best_model_HSIOnly_Deviation.h5")

In [7]:
with g.as_default():
    clf.load_weights(datasetName+"/best_model_HSIOnly_Deviation.h5")

ValueError: You are trying to load a weight file containing 0 layers into a model with 15 layers.